## EDA



### Distribution of data

-Visualization still missing

In [ ]:
# Import packages
import pandas as pd
import glob
from matplotlib import pyplot as plt
import os
import cv2
import random
import numpy as np

In [ ]:
# Load the dataframe names into a list
path_corrected = '../data/areas/corrected'
filenames_corrected = glob.glob(path_corrected + "/*.csv")
filenames_corrected.sort()

path_uncorrected = '../data/areas/uncorrected'
filenames_uncorrected = glob.glob(path_uncorrected + "/*.csv")
filenames_uncorrected.sort()

In [ ]:
# Load the dataframes into a list
dataframes_corr_list = []
for i in range(len(filenames_corrected)):
    temp_df = pd.read_csv(filenames_corrected[i])
    dataframes_corr_list.append(temp_df)

dataframes_uncorr_list = []
for i in range(len(filenames_uncorrected)):
    temp_df = pd.read_csv(filenames_uncorrected[i])
    dataframes_uncorr_list.append(temp_df)

In [ ]:
# Replace 1,2,3 with Fracture, Pore, Tiny Pore
for i in range(len(dataframes_uncorr_list)):    
    temp_df = dataframes_uncorr_list[i]
    temp_df['label_id'].replace([1,2,3], ['Fracture', 'Pore', 'Tiny Pore'], inplace=True)
    dataframes_uncorr_list[i] == temp_df

for i in range(len(dataframes_corr_list)):
    temp_df = dataframes_corr_list[i]
    temp_df['label_id'].replace([1,2,3], ['Fracture', 'Pore', 'Tiny Pore'], inplace=True)
    dataframes_corr_list[i] == temp_df



In [ ]:
# All elements of uncorrected list
all_elements = 0
elements_per_image = []
for i in range(len(dataframes_uncorr_list)):    
    temp_df = dataframes_uncorr_list[i]
    elements = len(temp_df)
    elements_per_image.append(elements)
    all_elements += elements

print(f'The total number of uncorrected elements are: {all_elements}')
print(f'The number of uncorrected elements per image are: {*elements_per_image,}')

In [ ]:
# All elements of corrected list
all_elements = 0
elements_per_image = []
for i in range(len(dataframes_corr_list)):    
    temp_df = dataframes_corr_list[i]
    elements = len(temp_df)
    elements_per_image.append(elements)
    all_elements += elements   

print(f'The total number of corrected elements are: {all_elements}')
print(f'The number of corrected elements per image are: {*elements_per_image,}')

In [ ]:
# Number of elements per class
total_frac_size = 0
total_pore_size = 0
total_tiny_pore_size = 0

for i in range(len(dataframes_corr_list)):    
    temp_df = dataframes_corr_list[i]
    temp_sizes = temp_df.groupby('label_id').count().reset_index()
    total_frac_size += temp_sizes.at[0, 'grain_id']
    total_pore_size += temp_sizes.at[1, 'grain_id']
    total_tiny_pore_size += temp_sizes.at[2, 'grain_id']

print(f'The total number of fractures are equal to {total_frac_size}')
print(f'The total number of pores are equal to {total_pore_size}')
print(f'The total number of tiny pores are equal to {total_tiny_pore_size}')

In [ ]:
# Total area for each class
# Number of elements per class
total_frac_area = 0
total_pore_area = 0
total_tiny_pore_area = 0

for i in range(len(dataframes_corr_list)):    
    temp_df = dataframes_corr_list[i]
    temp_sizes = temp_df.groupby('label_id').sum().reset_index()
    total_frac_area += temp_sizes.at[0, 'area']
    total_pore_area += temp_sizes.at[1, 'area']
    total_tiny_pore_area += temp_sizes.at[2, 'area']

print(f'The total area of fractures is equal to {round(total_frac_area/(total_frac_area+total_pore_area+total_tiny_pore_area)*100)}%')
print(f'The total area of pores is equal to {round(total_pore_area/(total_frac_area+total_pore_area+total_tiny_pore_area)*100)}%')
print(f'The total area of tiny pores is equal to {round(total_tiny_pore_area/(total_frac_area+total_pore_area+total_tiny_pore_area)*100)}%')



### Baseline model
The current approach employs a CNN for segmentation and a simple decision tree for labelling.
The labelling is then later corrected by hand. The difference between the original labelling and the corrections will give us an estimate of the current error (minus error introduced by human error).

In [ ]:
# Difference in area for both classes
frac_error = []
pore_error = []

for i in range(len(dataframes_corr_list)):
    # Get the area of the corrected images
    temp_df_corr = dataframes_corr_list[i]
    temp_sizes_corr = temp_df_corr.groupby('label_id').sum().reset_index()
    total_frac_area_corr = temp_sizes_corr.at[0, 'area']
    total_pore_area_corr = temp_sizes_corr.at[1, 'area']
    # Get the area of the uncorrected images
    temp_df_uncorr = dataframes_uncorr_list[i]
    temp_sizes_uncorr = temp_df_uncorr.groupby('label_id').sum().reset_index()
    total_frac_area_uncorr = temp_sizes_uncorr.at[0, 'area']
    total_pore_area_uncorr = temp_sizes_uncorr.at[1, 'area']
    # Get the difference 
    total_frac_difference = abs(total_frac_area_corr-total_frac_area_uncorr)/total_frac_area_uncorr
    total_pore_difference = abs(total_pore_area_corr-total_pore_area_uncorr)/total_pore_area_uncorr

    percent_frac_difference = round(total_frac_difference*100)
    percent_pore_difference = round(total_pore_difference*100)

    frac_error.append(percent_frac_difference)
    pore_error.append(percent_pore_difference)

print(f'The error of classification (%) for fractures within each image is equal to: {*frac_error,}')
print(f'The error of classification (%) for pores within each image is equal to: {*pore_error,}')

mean_error_frac = sum(frac_error)/len(frac_error)
mean_error_pore = sum(pore_error)/len(pore_error)

print(f'The mean area error for fractures is equal to {round(mean_error_frac)}%')
print(f'The mean area error for pores is equal to {round(mean_error_pore)}%')

In [ ]:
# Wrongly classified objects in total and percent
frac_error = []
pore_error = []

for i in range(len(dataframes_corr_list)):
    # Get the area of the corrected images
    temp_df_corr = dataframes_corr_list[i]
    temp_number_corr = temp_df_corr.groupby('label_id').count().reset_index()
    total_frac_corr = temp_number_corr.at[0, 'grain_id']
    total_pore_corr = temp_number_corr.at[1, 'grain_id']

    # Get the area of the uncorrected images
    temp_df_uncorr = dataframes_uncorr_list[i]
    temp_sizes_uncorr = temp_df_uncorr.groupby('label_id').count().reset_index()
    #print(temp_sizes_uncorr)
    total_frac_uncorr = temp_sizes_uncorr.at[0, 'grain_id']
    total_pore_uncorr = temp_sizes_uncorr.at[1, 'grain_id']
    # Get the difference 
    total_frac_difference = abs(total_frac_corr-total_frac_uncorr)/total_frac_uncorr
    total_pore_difference = abs(total_pore_corr-total_pore_uncorr)/total_pore_uncorr

    percent_frac_difference = round(total_frac_difference*100)
    percent_pore_difference = round(total_pore_difference*100)

    frac_error.append(percent_frac_difference)
    pore_error.append(percent_pore_difference)



In [ ]:
print(f'The error of classification (%) for fractures within each image is equal to: {*frac_error,}')
print(f'The error of classification (%) for pores within each image is equal to: {*pore_error,}')

mean_error_frac = sum(frac_error)/len(frac_error)
mean_error_pore = sum(pore_error)/len(pore_error)

print(f'The mean area error for fractures is equal to {round(mean_error_frac)}%')
print(f'The amount of mi... pores is equal to {round(mean_error_pore)}%')

### Data Image EDA

In [ ]:
# load images and sort masks and images
train_img_dir = '../data/data_train/train/images/train/'
train_mask_dir = '../data/data_train/train/masks/train/'

img_list = os.listdir(train_img_dir)
img_list.sort()
msk_list = os.listdir(train_mask_dir)
msk_list.sort()
num_images = len(os.listdir(train_img_dir))
print("Total number of training images are: ", num_images)

A total of 1020 image of size 512*512 exist for Training purposes

In [ ]:
# Plot of some images as a sanity check
img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 0)
mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot, cmap='gray')
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
print("Unique values in the mask are: ", np.unique(mask_for_plot))

There are four distinct grayscale values of the four different classes.